In [ ]:
from os.path import join, basename, dirname
import numpy as np
# local libraries
from cmftools import get_outlets, get_catchments

In [ ]:
map_dir = r'/home/dirk/Models/CaMa-Flood_v3.6.2_20140909/map/global_15min'
NE_dir = r'/home/dirk/Datasets/NaturalEarth'
# I/O
fn_nextxy = join(map_dir, "nextxy.tif")
fn_lonlat = join(map_dir, "lonlat.tif")
fn_outlets_all = join(map_dir, 'rivmth_all.txt')
# sample metadata from
names = ['uparea', "rivwth"]
sample_dict = {name: join(map_dir, "{}.tif".format(name)) for name in names}
sample_dict["dist2coast"] = join(NE_dir, 'ne_10m_dist2coast_eucl_1min.tif')
sample_dict

In [ ]:
# get outlets
outlets = get_outlets(fn_nextxy, fn_lonlat, sample_dict=sample_dict, fn_out=fn_outlets_all, res_str='15min')

outlets['dist2coast'] = np.floor(outlets['dist2coast'].values)
outlets['rivwth'] = np.floor(outlets['rivwth'].values)
print(len(outlets))
outlets.head()

In [ ]:
# filter
min_uparea=1e9 # 1000 km2
max_dist=10e3 # 10km
fn_outlets = join(map_dir, 'rivmth_upa{:.0e}_dist{:.0e}.txt'.format(min_uparea, max_dist))
print(fn_outlets)

outlets_select = outlets[np.logical_and(outlets['uparea'].values>min_uparea, outlets['dist2coast'].values<max_dist)]
outlets_select.to_csv(fn_outlets)
print(len(outlets_select))
outlets_select.head()

In [ ]:
# get basins tif
fn_catm = fn_outlets.replace('.txt', '_catm.tif')
print(fn_catm)
catm = get_catchments(fn_outlets, fn_nextxy, fn_catm)

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(catm)